# 02 Exercise Solution
---

# Visualising the Spatial Distribution of MSOA Accessibility to Green Space
---

Geospatial-statistical analysis of MSOA accessibility to publicly accessible green spaces in Greater London.

### National Statistics Postcode Lookup

* The [National Statistics Postcode Lookup (NSPL)](https://www.ons.gov.uk/methodology/geography/geographicalproducts/postcodeproducts) can be used to allocate your source statistics at postcode-level to a wide range of higher UK statistical and administrative geographies.
* The NSPL does this by allocating UK postcodes to Output Areas (OA). These OAs are then referenced to a wide range of higher statistical geographies (for example, local authority districts (LADs)) by a best-fit methodology that uses Census population data.
* The postcode centroid point geomety provides the 1-metre grid reference location (x, y) of the mean address in the postcode snapped to the nearest property.
* We'll use the NSPL for postcode centroid point geometries and for the references to Middle Layer Super Output Area (MSOA) and Regions.

### OS Open Greenspace

The OS OpenData product [OS Open Greenspace](https://www.ordnancesurvey.co.uk/business-government/products/open-map-greenspace) depicts the location and extent of spaces such as parks and sports facilities that are likely to be accessible to the public. Where appropriate, it also includes Access Points to show how people get into these sites. Its primary purpose is to enable members of the public to find and access greenspaces near them for exercise and recreation.

<img width="500"
     src="https://beta.ordnancesurvey.co.uk/img-assets/products/greenspace-open-london.x5201e7a5.jpg?w=1242&h=828&crop=828%2C828%2C207%2C0&f=webp?q=100&crop=2270,1422,0,0&w=1000"
     alt="OS Open Greenspace London"
     align="centre" />

### Middle Layer Super Output Areas

* [Middle Layer Super Output Areas (MSOAs)](https://geoportal.statistics.gov.uk/datasets/middle-layer-super-output-areas-december-2011-boundaries-generalised-clipped-bgc-ew-v3/explore) are an ONS census geography product comprised of nested OAs and LSOAs and providing a geogrpahical unit for census data releases.
* The postcodes in the NSPL are allocated to OAs by plotting each postcode's centroid directly into the OA boundaries. The postcodes are then allocated to the higher geographies that the postcode’s OA falls in by plotting the OA’s population weighted centroid into the digital boundary of the higher geography. A population weighted centroid is a grid reference that in a single summary point reflects the spatial distribution of the 2011 Census population in the OA. 

---

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from datetime import datetime
from folium import Map
from matplotlib.colors import ListedColormap
from shapely.geometry import box

### NSPL

1. Read the NSPL CSV data into a Pandas DataFrame.
2. Return the head of the DataFrame.
3. List all columns of the DataFrame.
4. Count the rows.
5. Subset the DataFrame only keeping live postcodes `(doterm IS NULL)` referencing the Greater London region `(rgn equal to E12000007)`.
6. Rename the following columns: `oseast1m` to `x`, `osnrth1m` to `x`, `lsoa11` to `lsoa11cd` and `rgn` to `rgn20cd`.
7. Count the rows in the subset.
8. Construct a GeoPandas GeoDataFrame from the Pandas DataFrame with the geometry column representing a point type constructed from the postcode centroid x and y coordinates.
9. Assign the GeoDataFrame CRS to `British National Grid (EPSG:27700)`.
10. Return the GeoDataFrame CRS assignment.
11. Count rows by geometry type.
12. Check for NULL postocde geometries.
10. Create a static plot of the subset GeoDataFrame. Use a qualitative colour hex value from the GDV toolkit and reduce the marker size and marker transparency to create a dot-denisty map.

In [ ]:
# Read the NSPL CSV data into a Pandas DataFrame
nspl = pd.read_csv(
    "../../data/office-for-national-statistics/nspl-nov-2023-uk.csv"
)

# Return the head of the DataFrame
nspl.head()

In [ ]:
# List all columns of the DataFrame
nspl.columns

In [ ]:
# Count the rows
# nspl.shape[0]
len(nspl)

In [ ]:
# Subset the DataFrame only keeping live postcodes referencing the Greater London region
nspl = nspl.loc[(nspl["doterm"].isnull()) & (nspl["rgn"] == "E12000007"),  ["pcds", "oseast1m", "osnrth1m", "msoa21", "rgn"]]

In [ ]:
# Rename columns
nspl = nspl.rename(
    columns={"oseast1m": "x", "osnrth1m": "y", "msoa21": "msoa21cd", "rgn": "rgn20cd"}
)

In [ ]:
nspl.head()

In [ ]:
# Count the rows in the subset
nspl.shape[0]

In [ ]:
# Construct a GeoPandas GeoDataFrame from the Pandas DataFrame with the geometry column representing a point type constructed from the postcode centroid x and y coordinates
nspl_gdf = gpd.GeoDataFrame(
    nspl,
    geometry=gpd.points_from_xy(
        x=nspl["x"],
        y=nspl["y"],
        # Assign British National Grid CRS
        # Unlike a GPKG there is no embedded CRS metadata in a CSV source
    ),
    
    crs="epsg:27700",
)

In [ ]:
# Return the GeoDataFrame CRS assignment
nspl_gdf.crs

In [ ]:
# Count rows by geometry type
nspl_gdf["geometry"].geom_type.value_counts()

In [ ]:
# Count NULL postcode geometries
nspl_gdf["geometry"].isna().sum()

In [ ]:
# Create a static plot of the subset GeoDataFrame
# Build on GDV best practice
f, ax = plt.subplots(figsize=(20,20))
ax = nspl_gdf.plot(color="#af58ba", alpha=0.2, markersize=1, ax=ax)
# Turn axis off
ax.set_axis_off()

### OS Open Greenspace

1. Read the `greenspace_site` layer from the GPKG dataset into a GeoPandas DataFrame.
2. Subset columns (keeping only `id`, `function` and `geometry`) and filter for green space function `Public Park Or Garden` or `Playing Field`.
3. Return the head of the GeoDataFrame.
4. Count the rows.
5. Count rows by geometry type.
6. Check geometry validity.
7. Create a static plot of the centroid of each MultiPolygon geometry. Use a qualitative colour hex value from the GDV toolkit.

In [ ]:
# Read the OS Open Greenspace GPKG data into a GeoPandas GeoDataFrame
osogs = gpd.read_file(
    filename="../../data/ordnance-survey/os-open-greenspace-gb.gpkg",
    # GPKG layer
    layer="greenspace_site",
)

# Subset columns and filter for green space function 'Public Park Or Garden' or 'Playing Field'
osogs = osogs.loc[
    osogs["function"].isin(["Public Park Or Garden", "Playing Field"]),
    ["id", "function", "geometry"],
]

# Return the head of the GeoDataFrame
osogs.head()

In [ ]:
# Count the rows
osogs.shape[0]

In [ ]:
# Count rows by geometry type
osogs["geometry"].geom_type.value_counts()

In [ ]:
# Check geometry validity
osogs["geometry"].is_valid.value_counts()

In [ ]:
# Create a static plot of the centroid of each MultiPolygon geometry
# Follow GDV best practice
ax = osogs["geometry"].centroid.plot(
    color="#00cd6c", alpha=0.2, markersize=1, figsize=(10, 20)
)
# Turn axis off
ax.set_axis_off()

### Spatially join the NSPL and OS Open Greenspace

1. Spatially join the rows within the nspl_gdf and the osogs GeoDataFrames where pairs of geometries are within 1000m.
    * Use the GeoPandas function ['sjoin_nearest'](https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.sjoin_nearest.html) to perform the spatial join.
    * Use a maximum join distance of 1000m.
    * Create a column called 'distance' which shows the distance between the pairs of geometries.
    * Ensure that every postcode in the nspl GeoDataFrame is retained within the join.
2. Return the head of the GeoDataframe.
3. Count the rows.
4. Count the number of unique postcodes - what can you gleen by comparing against the total row count?
5. Check that the number of unique postcodes is equal to the number of postcodes in Greater London in the subset of the NSPL.
6. Calculate the minimum distance to a green space per postcode. Do this by grouping by the pcds and msoa21cd columns and evaluating the minimum distance.
7. Return the head of the DataFrame.
8. Calculate the mean distance to a green space per MSOA. Do this by grouping by the msoa21cd column of the minimum distance per postcode DataFrame in #6 above and evaluating the mean distance.
9. Return head of DataFrame.
10. Return DataFrame descriptive statistics.
11. Plot a MSOA-distance histogram.
12. Populate a new 'rank' column binning each MSOA into one of seven classes ordering by the mean distance.
13. Return head of DataFrame.
14. Count the rows.
15. Check that the number of MSOAs is equal to the number of unique MSOAs in Greater London in the subset of the NSPL.

In [ ]:
# Spatially join the rows within the nspl_gdf and the osogs GeoDataFrames where pairs of geometries are within 1000m
nspl2osogs = gpd.sjoin_nearest(
    left_df=nspl_gdf,
    right_df=osogs,
    how="left",
    max_distance=1000,  # Maximum search radius
    distance_col="distance",
)

# Return the head of the GeoDataframe
nspl2osogs.head()

In [ ]:
# Count the rows
nspl2osogs.shape[0]

In [ ]:
# Count the number of unique postcodes
nspl2osogs["pcds"].nunique()

In [ ]:
# Check that the number of unique postcodes is equal to the number of postcodes in Greater London in the subset of the NSPL
nspl2osogs["pcds"].nunique() == nspl_gdf.shape[0]

In [ ]:
# Calculate the minimum distance to a green space per postcode
# Do this by grouping by the pcds and msoa21cd columns and evaluating the minimum distance
pcds_min = nspl2osogs.groupby(["pcds", "msoa21cd"])["distance"].min().reset_index()

# Return the head of DataFrame
pcds_min.head()

In [ ]:
pcds_min.shape[0]

In [ ]:
pcds_min[pcds_min["distance"].isnull()]

In [ ]:
pcds_min.fillna({"distance": 1000}, inplace=True)

In [ ]:
# Calculate the mean distance to a green space per MSOA
# Do this by grouping by the msoa21cd column of the distance_gdf DataFrame and evaluating the mean distance
msoa_mean = pcds_min.groupby(["msoa21cd"])["distance"].mean().reset_index()

# Return the head of DataFrame
msoa_mean.head()

In [ ]:
# Return DataFrame descriptive statistics
msoa_mean.describe()

In [ ]:
# Create figure and axes objects
f, ax = plt.subplots(figsize=(10, 5))

# Set x label
ax.set_xlabel("Mean Distance (m)")
# Set y label
ax.set_ylabel("Frequency")
# Set title
ax.set_title("Distribution of Mean MSOA Postcode to Greenspace Distances")

# Plot a MSOA-mean distance histogram
_ = ax.hist(msoa_mean["distance"], bins=50)

In [ ]:
# Populate a new 'rank' column binning each MSOA into one of seven classes ordering by the mean distance
# Specify rank labels 1-7 equal to number of colour values in GDV sequential palette
rank_labels = list(range(1, 8))

# Populate rank column
msoa_mean["rank"] = pd.qcut(msoa_mean["distance"], q=7, labels=rank_labels)

# Return head of DataFrame
msoa_mean.head()

In [ ]:
# Count the rows
msoa_mean.shape[0]

In [ ]:
# Check that the number of MSOAs is equal to the number of unique MSOAs in Greater London in the subset of the NSPL
assert msoa_mean.shape[0] == nspl["msoa21cd"].nunique()

### MSOA Boundaries

1. Read the MSOA GeoPackage (GPKG) dataset into a GeoPandas DataFrame.
2. Return the head of the GeoDataFrame.
3. Count the rows.
4. Subset the columns keeping only `MSOA21` and `geometry`.
5. Rename the `MSOA21` column to `msoa21cd`.
6. Count rows by geometry type.
7. Check geometry validity.
8. Inner join (by attribute) the rows within the lsoa GeoDataFrame and the lsoa_mean DataFrame on the `msoa21cd` column.
9. Return the head of the GeoDataFrame
10. Count the rows.

In [ ]:
# Read the MSOA GPKG data into a GeoPandas GeoDataFrame
msoa = gpd.read_file(
    filename="../../data/office-for-national-statistics/msoa-2021-boundaries-bfc-ew.gpkg"
)

# Return the head of GeoDataFrame
msoa.head()

In [ ]:
# Count the rows
msoa.shape[0]

In [ ]:
# Subset columns
msoa = msoa[["MSOA21CD", "geometry"]]

In [ ]:
# Rename columns
msoa = msoa.rename(columns={"MSOA21CD": "msoa21cd"})

In [ ]:
# Count rows by geometry type
msoa["geometry"].geom_type.value_counts()

In [ ]:
# Check geometry validity
msoa["geometry"].is_valid.value_counts()

In [ ]:
# Inner join (by attribute) the rows within the lsoa GeoDataFrame and the lsoa_mean DataFrame on the `msoa21cd` column
# GeoDataFrame required to be in the 'left' argument to preserve a GeoDataFrame output
msoa_mean_gdf = pd.merge(
    msoa, msoa_mean, how="inner", on="msoa21cd", suffixes=("_l", "_r")
)

# Return the head of GeoDataFrame
msoa_mean_gdf.head()

In [ ]:
# Count the rows
msoa_mean_gdf.shape[0]

### OS GeoDataViz Sequential Palette

1. Download and decode the JSON representation of the OS [GDV colour palettes](https://raw.githubusercontent.com/OrdnanceSurvey/GeoDataViz-Toolkit/master/Colours/GDV-colour-palettes-v0.7.json) via a Requests HTTP GET request.
2. Access the `m2` colour palette within the `sequential` palette group.
3. Create a Matplotlib [ListedColormap](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.ListedColormap.html) colour map from the m2 palette.
4. Return the colour map.

In [ ]:
# Download and decode the JSON representation of the OS GDV colour palettes via a Requests HTTP GET request
gdv = "https://raw.githubusercontent.com/OrdnanceSurvey/GeoDataViz-Toolkit/master/Colours/GDV-colour-palettes.json"
# Turn off SSL certificate verification
gdv_json = requests.get(gdv, verify=False).json()

# Access the 'm2' colour palette within the 'sequential' palette group
# ['#FCE1A4', '#FABF7B', '#F08F6E', '#E05C5C', '#D12959', '#AB1866', '#6E005F']
m2 = gdv_json["sequential"]["m2"]

# Create a Matplotlib ListedColormap colour map from the m2 palette
gdv_cmap = ListedColormap(colors=m2, name="gdv_seq_m2_cmap")

# Return the colour maps
gdv_cmap

### Static Map

1. Create a GeoPandas static plot overlaying the centroids of the `osogs` GeoDataFrame, within the Greater London bounding box only, ontop of a choropleth map showing MSOA mean distance to green space, from the `msoa_mean_gdf` GeoDataFrame. The map should be styled using GDV best practice and the mean distance coloured using the `m2` GDV colour map against the `rank` bin column. 

In [ ]:
# Create figure and axes objects
f, ax = plt.subplots(figsize=(20, 20))

# Set title
ax.set_title("Greater London Authority MSOA Accessibilty To Green Space")
# Turn axis off
ax.set_axis_off()

# Plot the msoa_mean_gdf GeoDataFrame
msoa_mean_gdf.plot(
    ax=ax,
    column='rank',
    categorical=True,
#     column="distance",
#     scheme="Quantiles",  # mapclassify classification scheme
#     k=7,  # 7 bins
    cmap=gdv_cmap.reversed(),  # GDV Matplotlib colour map
    linewidth=0.1,
    edgecolor="#ffffff",
    legend=True,
)

# Clip the osogs GeoDataFrame to the bounding box of Greater London
osogs = osogs.cx[503568.1996:561957.4962, 155850.7975:200933.9026]

# Plot the centroid each geometry in the osogs GeoDataFrame
osogs["geometry"].centroid.plot(ax=ax, color="#06592a", alpha=0.75, markersize=2)

### Interactive Map

1. Create a GeoPandas and leaflet/folium interactive choropleth map showing MSOA mean distance to green space, from the `msoa_mean_gdf` GeoDataFrame. The map should be styled using GDV best practice and the mean distance coloured using the `m2` GDV colour map against the `rank` bin column. The folium map should use a map instance with an OS Maps API Light Style 3857 base map.

In [ ]:
# Provide OS Maps API layer name
layer = "Light_3857"
# Insert OS Data Hub project API key
key = "frKhvBUiMB5DGwl3pGb2GzcOz6ApgyP0"

# OS Data Hub base path - https://api.os.uk
# OS Maps API ZXY end point path - /maps/raster/v1/zxy/
url = f"https://api.os.uk/maps/raster/v1/zxy/{layer}/{{z}}/{{x}}/{{y}}.png?key={key}"

In [ ]:
# Return centroid of MSOA Greater London Authority BBOX in WGS-84 CRS
c = (
    gpd.GeoSeries(box(*msoa_mean_gdf.total_bounds))
    .centroid.set_crs(crs="epsg:27700")
    .to_crs(crs="epsg:4326")
)

In [ ]:
# Create Folium map with OS Maps API base map
m = Map(
    location=[
        c.y,
        c.x,
    ],  # Map centre coordinates (by convention latitude (y), longitude (x))
    tiles=url,
    attr=f"Contains OS data &copy; Crown copyright and database rights {datetime.now().year}",  # OS Data Hub attribution statement
    min_zoom=7,  # See EPSG:3857 Tile Matrix Set - https://osdatahub.os.uk/docs/wmts/technicalSpecification
    max_zoom=16,
    zoom_start=10,
)

In [ ]:
# Create an interactive leaflet map based on the 'msoa_mean_gdf' GeoDataFrame
# Apply GDV best practice
msoa_mean_gdf.explore(
    column="rank",  # Qualitative styling based on 'rank' column values
    cmap=gdv_cmap,  # Use GDV 'm2' colour map
    m=m,  # folium Map created above
    style_kwds={"color": "#fff", "weight": 0.05},
)  # Edit stroke styling

---

### Extension: Quantifying accessibilty via a different metric

Finding the distance to each postcode's nearest greenspace isn't the only way to quantify accessibility to green space.  Considering that the area (utlity) of a green space site may be an important factor in conjunction with distance, one alterative could be to measure the total area of green space within a certain distance of each postcode.

1. Create a new GeoDataFrame by buffering the NSPL postcode POINT geometries in the `nspl_gdf` GeoDataFrame by a threshold distance.
2. Add a new column to the GeoDataFrame representing the area of the new POLYGON geometry for each postcode.
3. Return the head of the GeoDataFrame.

In [ ]:
# Distance (buffer) threshold in metres
distance_threshold = 500

# Copy the NSPL GeoDataFrame
pcds_buffer = nspl_gdf
# Buffer the postcode POINT geometries by the threshold distance
# This transforms the geometry type from POINT -> POLYGON
pcds_buffer["geometry"] = pcds_buffer.buffer(distance_threshold)

# Add a new column representing the area of the new POLYGON geometry for each postcode
pcds_buffer["threshold_area"] = pcds_buffer.area

# Return the head of the GeoDataFrame
pcds_buffer.head()

4. Create a new GeoDataFrame by evaluating the intersection (shared geometry) between the buffered postcode POLYGON geometries and the OS Open GreenSpace data in the `osogs` GeoDataFrame.
5. Dissolve (spatially union) the intersection geometries by postcode.
6. Add a new column to the GeoDataFrame representing the area of the intersection geometry by postcode.
7. Add a new column to the GeoDataFrame representing a normalised area metric - divide the intersection geometry area by the area of the buffered postcode geometry.
8. Return the head of the GeoDataFrame.

In [ ]:
# Levergage GeoPandas overlay to evaluate the intersection (shared geometry) between the buffered postcode
# POLYGON geometries and the OS Open GreenSpace data in the `osogs` GeoDataFrame
pcds_buffer_osog_int = pcds_buffer.overlay(osogs, how="intersection")

# Dissolve (spatially union) the intersection geometries by postcode.
pcds_buffer_osog_int = pcds_buffer_osog_int.dissolve(by="pcds").reset_index()

# Add a new column to the GeoDataFrame representing the area of the intersection geometry by postcode
pcds_buffer_osog_int["area"] = pcds_buffer_osog_int.area

# Add a new column to the GeoDataFrame representing a normalised area metric -
# divide the intersection geometry area by the area of the buffered postcode geometry
pcds_buffer_osog_int["area_normalised"] = (
    pcds_buffer_osog_int["area"] / pcds_buffer_osog_int["threshold_area"]
)

# Return the head of the GeoDataFrame
pcds_buffer_osog_int.head()

9. A subset of postcodes will not be represented in the data because the buffered POLYGON geometry does not intersect any of the OS Open GreenSpace feature geometries. Add the missing subset of postcodes to the data setting the normalised area metric to a value of 0.
10. Return the head of the GeoDataFrame.

In [ ]:
# Add the missing subset of postcodes to the data setting the normalised area metric to a value of 0
pcds_buffer_osog_int = pcds_buffer.merge(
    pcds_buffer_osog_int[["pcds", "area_normalised"]], on="pcds", how="left"
)
pcds_buffer_osog_int["area_normalised"] = pcds_buffer_osog_int[
    "area_normalised"
].fillna(0)

# Return the head of the GeoDataFrame
pcds_buffer_osog_int.head()

11. Test whether the number of rows in the NSPL and postcode normalised area datasets are equal.

In [ ]:
# Test whether the number of rows in the NSPL and postcode normalised area datasets are equal
assert nspl.shape[0] == pcds_buffer_osog_int.shape[0]

12. Plot a histogram of the distribution of postcode normalised area values.

In [ ]:
# Plot a histogram of the distribution of postcode normalised area values
# The normalised area metric is very biased towards small values
# Right-skewed (long right tail)
pcds_buffer_osog_int.hist("area_normalised", bins=50, figsize=(8, 4))

13. Summarise the postcode normalised area at MSOA-level by calculating the mean area by MSOA.

In [ ]:
# Summarise the postcode normalised area at MSOA-level by calculating the mean area by MSOA
# Group by msoa21cd key
msoa_area = (
    pcds_buffer_osog_int.groupby(["msoa21cd"])["area_normalised"].mean().reset_index()
)
msoa_area_gdf = pd.merge(
    msoa, msoa_area, how="inner", on="msoa21cd", suffixes=("_l", "_r")
)

14. Create a static GeoDataFrame plot representing the spatial distribution in MSOA mean postcode normalised area. Bin the data into seven equal bins and style the classes using a sequential OS GDV colour palette. Overlay the source OS Open GreenSpace MULTIPOLYGON feature data ontop of the thematic map.

In [ ]:
# Create figure and axes objects
fig, ax = plt.subplots(figsize=(20, 20))

# Plot the msoa_mean_gdf GeoDataFrame
msoa_area_gdf.plot(
    ax=ax,
    column="area_normalised",
    scheme="Quantiles",  # mapclassify binning scheme
    k=7,  # Number of bins for mapclassify scheme
    # Reverse the colour map, because in this case smaller values correspond to increased accessibilty
    cmap=gdv_cmap.reversed(),
    linewidth=0.1,
    edgecolor="#fff",
    legend=True,
)

# Clip the osogs GeoDataFrame to the bounding box of Greater London
osogs = osogs.cx[503568.1996:561957.4962, 155850.7975:200933.9026]

# Since this metric accounts for greenspace area (not just location), we might want to also plot the full MULTIPOLYGON extent
# of greenspaces
osogs.plot(ax=ax, color="#06592a", edgecolor="none", alpha=0.1)  # Transparent faces
osogs.plot(
    ax=ax, color="none", edgecolor="#06592a", alpha=0.75
)  # Less transparent boundaries

# Turn axis off
ax.set_axis_off()